---

# CSCI 3202, Spring 2022
# Homework 5
# Due: Friday April 15, 2022 at 6:00 PM

<br> 

### Your name:

<br> 

---

Some useful packages and libraries:




In [102]:
from scipy import stats
import unittest
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string

---

# Hidden Markov Models
In this assignment, you will build a Hidden Markov Model from a data set to detect typos in text. One major difference between this assignment and previous assignments in this class is we're not providing any skeleton code, you need to generate your code from scratch. You can use any code provided during lecture or on previous assignments. 
<p><strong>Please read the entire assignment before beginning.</strong> 
    
<p>The main components of the assignment are the following:

### Part I - Building an HMM from data
1.  Implement a method to parse the data file and build an HMM from the data.
2.  Implement a method, Laplace smoothing, to smooth the transition and emission probabilities

### Part II - Implement the Viterbi algorithm
1.	Implement the Viterbi algorithm for finding the most likely sequence of states through the HMM, given "evidence"; and
2.	Run your code on a separate dataset and explore its performance.
    
### Part III - Summarize your results
    



## Part 1 - Building an HMM from data
The first part of the assignment is to build an HMM from data. Recall that an HMM involves a hidden state that changes over time, as well as observable evidence that is used to infer the hidden state. In our example from lecture, we used the observation of how a patient felt to infer whether they were healthy or sick. 

An HMM is defined by three sets of probabilities:
1.	For each state s, the probability of observing each output o at that state (P(E[t]=o | X[t]=s). These are the emission probabilities.
2.	From each state s, the probability of traversing to every other state s' in one time step (P(X[t+1]=s' | X[t]=s)). These are the transition probabilities.
3.	A distribution over the start state (P(X[0])). This is the initial state distribution.

For the start state distribution (P(X[0])), in this assignment, you will assume that there is a single dummy start state, distinct from all other states, and to which the HMM can never return.  When you implement the Viterbi algorithm in Part III, you will need to include the probability of making a transition from this dummy start state to each of the other states.

### Generate the conditional probability tables
To generate the transition and emission probability tables, use the data file called <em>typos20.data</em> on Canvas that contains sequences of state-output pairs, i.e., sequences of the form: <br>

x[1] e[1] <br>
x[2] e[2] <br>
. <br>
. <br>
. <br>
x[n] e[n]. <br>  

In this data, the x[i] is the state and the e[i] is the observation at time i. 

For instance, to estimate the probability of output o being observed in state s, you count the number of times that output o appears with state s in the given data, and divide by a normalization constant (so that the probabilities of all outputs from that state add up to one).  In this case, that normalization constant would simply be the number of times that state s appears at all in the data.

In this problem, state is letter that should have been typed, and output is the letter that was typed.  Given a sequence of outputs (i.e., actually typed letters), the problem is to reconstruct the hidden state sequence (i.e., the intended sequence of letters).  Thus, data for this problem looks like this:

i i <br>
n n <br>
t t <br>
r r <br>
o o <br>
d x <br>
u u <br>
c c <br>
t t <br>
i i <br>
o i <br>
n n <br>
_ _ <br>
t t <br>
h h <br>
e e <br>
_ _ <br>

where the left column is the correct text and the right column contains text with errors.

### Data
Data for this assignment was generated by starting with a text document, in this case, the Unabomber's Manifesto, which was chosen not for political reasons, but for its convenience being available on-line and of about the right length, converting all numbers and punctuation to white space and converting all letters to lower case.  The remaining text is a sequence only over the lower case letters and the space character, represented in the data files by an underscore character.  Next, typos were artificially added to the data as follows: with 80% probability, the correct letter is transcribed, but with 20% probability, a randomly chosen neighbor (on an ordinary physical keyboard) of the letter is transcribed instead.  Space characters are always transcribed correctly. 

As an example, the original document begins:

<em>introduction the industrial revolution and its consequences have been a disaster for the human race they have greatly increased the life expectancy of those of us who live in advanced countries but they have destabilized society have made life unfulfilling have subjected human beings to indignities have led to widespread psychological suffering in the third world to physical suffering as well and have inflicted severe damage on the natural world the continued development of technology will worsen the situation it will certainly subject human beings to greater indignities and inflict greater damage on the natural world it will probably lead to greater social disruption and psychological suffering and it may lead to increased physical suffering even in advanced countries the industrial technological system may survive or it may break down if it survives it may eventually achieve a low level of physical and psychological suffering but only after passing through a long and very painful period of adjustment and only at the cost of permanently reducing human beings and many other living organisms to engineered products and mere cogs in the social machine </em>

With 20% noise, it looks like this:

<em>introductipn the industfial revolhtjon and its consequences bafw newn a diszster rkr the yumab race thdy have grwatky increased the ljte esoectandy od thosr of is who libe in advanced coubfries but they have fewtabipuzee xociwty have made life ujfuorillkng have wubjwdted humah beints to incihbjtids have led to qidespreze lsyxhllotical shffeding kn tne third wkrld to phyxicql sufcefimg as weol and hqve ingoidtex srvere damsge on the natural world the confinued developmeng of twvhjllogy will wotsen thd situation it wull certaknly sunjrct yyman beingw tl greater ibdignities snd infpixt greagwr damsge on fhe natural alrld it wjlk probably lwad tk grezter sofiqp disrupgiln and pstchokofucal wufterkng anc it may kead fl uncreqxed pgusiczl sucfreinh even in acgajved countries the indhsteial tedhnologicak system may survivr or ut nay brezk down uf it survives it nay evenyuakly achieve a los lwvel of phyxkcal and psycyoligical sufveribg but only after passing theough a long amd very painful periox od adjuwtmebt and only at the fost kf permsnently reducing hymaj veings abs nsjy otgwr kuving orbanisms to envineered leoduxfs amd mere clgs in thr soxiap maxhjne </em>

The error rate (fraction of characters that are mistyped) is about 16.5% (less than 20% because space characters were not corrupted).

### Implement Laplace smoothing
When you generate the transition and emission probabilities, you will also need to smooth the estimates.  Here's the rationale, consider flipping a coin for which the probability of heads is p, where p is unknown, and our goal is to estimate p. We count how many times the coin comes up heads and divide by the total number of coin flips.  If we flip the coin 1000 times and it comes up heads 367 times, it is very reasonable to estimate p as approximately 0.367.  However, suppose we flip the coin only twice and we get heads both times.  Is it reasonable to estimate p as 1.0?  Intuitively, given that we only flipped the coin twice, we don’t have enough data to conclude that the coin will always come up heads. Smoothing is a way of avoiding these types of conclusions when we have limited data available.  

A simple smoothing method, called Laplace smoothing (or Laplace's law of succession or add-one smoothing in your textbook), is to estimate p by:

<p>
<center>$\frac{(1+number of heads)}{(2+number of flips)}$</center>

We add 1 to each output that could be observed in the numerator, and add the total number of states to the denominator.

If we are keeping count of the number of heads and the number of tails, this rule is equivalent to starting each of our counts at one, rather than zero.  This latter view generalizes to the case in which there are more than two possible outcomes (for instance, estimating the probability of a die coming up on each of its six faces). 
Another advantage of Laplace smoothing is that it avoids estimating any probabilities to be zero, even for events never observed in the data.  For HMMs, this is important since zero probabilities can be problematic for some algorithms.
    
For this assignment, you should use Laplace-smoothed estimates of probabilities.  For instance, returning to the problem of estimating the probability of output o being observed in state s, you would use one plus the number of times output o appears in state s in the given data, divided by a normalization constant. In this case, the normalization constant would be the number of times state s appears in the data, plus the total number of possible outputs. You will need to also work out Laplace-smoothed estimates for item 2, i.e., for the probability of making a transition from one state to another, as well as the probability of making a transition from the dummy start state to any of the other states. 


In [147]:
## Your code goes here.
## Your code needs to display the first 10 lines of your emission and transition tables, 
## and generate an output file with the complete tables, called Assignment8Output_Lastname_Firstname
## Your code also needs to be structured as an HMM class


class HMM:
    #creats out transition dictionary (matrix)
    def Transition(data):
        #a dictionary of dictionarys storing probility (row, colum)
        count = {}

        #creating keys for count dictionary (rows)
        for i in string.ascii_lowercase[:26]:
            count[i] = {}
        count["_"] = {}

        #creating keys for count subdictionary  (columns)
        for i in count.keys():
            for j in string.ascii_lowercase[:26]:
                count[i][j] = 0
            count[i]["_"] = 0

        #adding values
        for i in range(0, len(data)-1):
            count[data.iloc[i,0][0]][data.iloc[i+1,0][0]] += 1
        
        #laplase smoothing
        for i in string.ascii_lowercase[:26]:
            Icount = 0
            for j in string.ascii_lowercase[:26]:
                Icount += count[i][j]
            Icount += count[i]["_"]

            for j in string.ascii_lowercase[:26]:
                count[i][j] = (count[i][j] + 1)/(Icount + 27)
            count[i]["_"] = (count[i]["_"] + 1)/(Icount + 27)
        
        Icount = 0
        for j in string.ascii_lowercase[:26]:
            Icount += count["_"][j]
        Icount += count["_"]["_"]

        for j in string.ascii_lowercase[:26]:
            count["_"][j] = (count["_"][j] + 1)/(Icount + 27)
        count["_"]["_"] = (count["_"]["_"] + 1)/(Icount + 27)

        return count


    #creats out emissions dictionary (matrix)
    def Emissions(data):
        #a dictionary of dictionarys storing probility (row, colum)
        count = {}

        #creating keys for count dictionary (rows)
        for i in string.ascii_lowercase[:26]:
            count[i] = {}
        count["_"] = {}

        #creating keys for count subdictionary  (columns)
        for i in count.keys():
            for j in string.ascii_lowercase[:26]:
                count[i][j] = 0
            count[i]["_"] = 0

        #adding values
        for i in range(0, len(data)-1):
            count[data.iloc[i,0][0]][data.iloc[i,0][2]] += 1
        
        #laplase smoothing
        for i in string.ascii_lowercase[:26]:
            Icount = 0
            for j in string.ascii_lowercase[:26]:
                Icount += count[i][j]
            Icount += count[i]["_"]

            for j in string.ascii_lowercase[:26]:
                count[i][j] = (count[i][j] + 1)/(Icount + 27)
            count[i]["_"] = (count[i]["_"] + 1)/(Icount + 27)
        
        Icount = 0
        for j in string.ascii_lowercase[:26]:
            Icount += count["_"][j]
        Icount += count["_"]["_"]

        for j in string.ascii_lowercase[:26]:
            count["_"][j] = (count["_"][j] + 1)/(Icount + 27)
        count["_"]["_"] = (count["_"]["_"] + 1)/(Icount + 27)

        return count

    def __init__(self, data):
        self.data = pd.read_csv('typos20Test.data', sep="\n")
        self.Transition = Transition(self.data)
        self.Emissions = Emissions(self.data)


data = pd.read_csv('typos20.data', sep="\n")


G = HMM(data)

G.Transition


{'a': {'a': 0.0007874015748031496,
  'b': 0.024409448818897637,
  'c': 0.05511811023622047,
  'd': 0.025196850393700787,
  'e': 0.0007874015748031496,
  'f': 0.007874015748031496,
  'g': 0.01889763779527559,
  'h': 0.0015748031496062992,
  'i': 0.030708661417322834,
  'j': 0.0031496062992125984,
  'k': 0.025196850393700787,
  'l': 0.15039370078740158,
  'm': 0.02125984251968504,
  'n': 0.17795275590551182,
  'o': 0.0007874015748031496,
  'p': 0.011811023622047244,
  'q': 0.0007874015748031496,
  'r': 0.06850393700787402,
  's': 0.07165354330708662,
  't': 0.16692913385826771,
  'u': 0.014960629921259842,
  'v': 0.031496062992125984,
  'w': 0.002362204724409449,
  'x': 0.0007874015748031496,
  'y': 0.024409448818897637,
  'z': 0.0015748031496062992,
  '_': 0.06062992125984252},
 'b': {'a': 0.0199203187250996,
  'b': 0.00398406374501992,
  'c': 0.00398406374501992,
  'd': 0.00398406374501992,
  'e': 0.3386454183266932,
  'f': 0.00398406374501992,
  'g': 0.00398406374501992,
  'h': 0.0039


## Part II - Viterbi - finding the most likely sequence

Once your CPTs for your HMM are built, the second part of the assignment is to implement the Viterbi algorithm and use your CPTs to compute the most probable sequence of states (according to the HMM that you built from the data) for a given sequence of outputs.

There is another file on Moodle called *typos20Test.data* that contains test sequences of state-output pairs. You will provide your Viterbi code with just the output part of each of these sequences, and from this, you must compute the most likely sequence of states to produce the output sequence. The state part of these sequences is provided so that you can compare the estimated state sequences generated by your code to the actual state sequences that generated this data.  Note that these two sequences will not necessarily be identical, even if you have correctly implemented the Viterbi algorithm.

A numerical tip: the Viterbi algorithm involves multiplying many probabilities together.  Since each of these numbers is smaller than one (possibly much smaller), you can end up working with numbers that are tiny enough to be computationally indistinguishable from zero. To avoid this problem, it is recommended that you work with log probabilities. To do so, rather than multiplying two probabilities p and q, simply add their logarithms using the rule:
<p>
<center> log(pq) = log(p) + log(q) </center>

You will probably find that there is never a need to store or manipulate actual probabilities; instead, everything can be done using log probabilities.
The text reconstructed using an HMM with the Viterbi algorithm looks like this:
    <p>
*introduction the industrial revoluthon and its consequences bare neen a dissster ror the tuman race they have greatly increased the lite esoectandy od those of is who libe in advanced counfries but they have festabupusee cocisty have made live intiorilling have wibjested human beints to incingitids have led to widesprese lsysullotical suffeding in the third world to physical surcefing as weol and have ingoistes severe damage on the natural world the continued developmeng of techillogy will wotsen the situation it will certaknly sunirct tyman beinge tl greater indithities and infoist greager damage on the natural aleld it will probably owad to grester sofial distuption and pstchomofucal wiftering and it may kead fl increqxed ogusical suctreing even in achanved countries the industeial technologicak system may survive or ut nay break down if it survives it nay eventually achieve a los level of physical and psycholigical survering but only arter passing theough a long and very paindul perios od adjustment and only at the fost of permanently reducing human veings ans nany other kiving organisms to envineered leodusts and mere clys in the social machine*
        
The error rate has dropped to about 10.4%.


In [141]:
## Your code goes here. You need to implement the viterbi algorithm, and display the output that your code generates
## You also need to display the error rate for your algorithm.
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    for st in states:
        V[0] [st] = {"prob": start_p[st] * emit_p[st] [obs[0]], "prev": None}
    # Run Viterbi when t > 0
    for t in range(1, len(obs)):
        V.append({})
        for st in states:
            max_tr_prob = V[t - 1] [states[0]] ["prob"] * trans_p[states[0]] [st]
            prev_st_selected = states[0]
            for prev_st in states[1:]:
                tr_prob = V[t - 1] [prev_st] ["prob"] * trans_p[prev_st] [st]
                if tr_prob > max_tr_prob:
                    max_tr_prob = tr_prob
                    prev_st_selected = prev_st

            max_prob = max_tr_prob * emit_p[st] [obs[t]]
            V[t] [st] = {"prob": max_prob, "prev": prev_st_selected}

    for line in dptable(V):
        print(line)

    opt = []
    max_prob = 0.0
    best_st = None
    # Get most probable state and its backtrack
    for st, data in V[-1].items():
        if data["prob"] > max_prob:
            max_prob = data["prob"]
            best_st = st
    opt.append(best_st)
    previous = best_st

    # Follow the backtrack till the first observation
    for t in range(len(V) - 2, -1, -1):
        opt.insert(0, V[t + 1] [previous] ["prev"])
        previous = V[t + 1] [previous] ["prev"]

    print ("The steps of states are " + " ".join(opt) + " with highest probability of %s" % max_prob)

def dptable(V):
    # Print a table of steps from dictionary
    yield " " * 5 + "     ".join(("%3d" % i) for i in range(len(V)))
    for state in V[0]:
        yield "%.7s: " % state + " ".join("%.7s" % ("%lf" % v[state] ["prob"]) for v in V)


viterbi("thr", string.ascii_lowercase[:26] + "_", )

'abcdefghijklmnopqrstuvwxyz_'

## Part III - Summarize your results
As part of this assignment, include a short report in your notebook describing your work. The report should contain the following:
### Purpose: 
Purpose should be clear. It doesn't need to be long, but it should be clear.
### Procedure:
##### Transition Probabilities
In this section explain the way you have generated the Transition probabilities and the data structure you used.
##### Emission Probabilities
In this section explain the way you have generated the Emission probabilities and the data structure you used.
##### Viterbi Algorithm
In this section explain about the algorithm and the data structure you used for storing the values in the matrix with its back pointer.
### Data
Explain about the data you used, including the data included in the data set and any pre-processing you did to the data.
### Results
Explain your observations, including the sentences that you tested and their corresponding outputs. Were there any sentences where your algorithm did not work?


Your report goes here.
